# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
from transformers import AutoTokenizer #, AutoModelForSequenceClassification

In [6]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [23]:
import importlib
importlib.reload(util_elastic)

<module 'util.util_elastic' from '/home/borela/fontes/ind-ir/code/util/util_elastic.py'>

In [8]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [9]:
PATH_QUERY = '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'
PATH_DOC =  '../data/juris_tcu_index/doc.csv'

In [10]:
%%time
import logging
logging.getLogger("haystack").setLevel(logging.WARNING) #WARNING, INFO

CPU times: user 211 µs, sys: 0 ns, total: 211 µs
Wall time: 214 µs


# Data load

In [11]:
df_doc = pd.read_csv(PATH_DOC)

In [12]:
df_doc.shape

(13252, 28)

In [13]:
df_doc.head()

,TEXT,ID,NAME,DATE_REFERENCE,CLASS,HAS_SCOPE_NOTE,HAS_DEFINITION,HAS_SYNONYM,HAS_EXAMPLE,HAS_ENGLISH_TRANSLATION,HAS_SPECIALIZATION,HAS_RELATED_TERM,HAS_SPANISH_TRANSLATION,HAS_GENERALIZATION,TEXT_DEFINITION,TEXT_SYNONYM,TEXT_RELATED_TERM,TEXT_SCOPE_NOTE,TEXT_EXAMPLE,TEXT_ENGLISH_TRANSLATION,TEXT_SPANISH_TRANSLATION,TEXT_SPECIALIZATION,TEXT_GENERALIZATION,COUNT_INDEX_AREA,COUNT_INDEX_THEME,COUNT_INDEX_SUBTHEME,COUNT_INDEX_EXTRA,COUNT_INDEX_TOTAL
0,"O termo é ""Abandono de cargo"".\nAbandono de cargo tem definição: ""Configura abandono de cargo a ...",1,Abandono de cargo,20230605,Termo,1,1,1,0,1,0,1,1,0,Configura abandono de cargo a ausência intencional do servidor ao serviço por mais de trinta dia...,Abandono de função e Abandono do cargo.,"Abandono de emprego, Deserção, Falta grave, Função pública, Demissão de servidor, Inassiduidade ...",Artigo 323 Código Penal,NaN,Abandonment of office.,Abandono de puesto.,NaN,NaN,0,1,0,0,1
1,"O termo é ""Abastecimento de tropas e meios"".\nAbastecimento de tropas e meios é uma especializaç...",2,Abastecimento de tropas e meios,20230605,Termo,0,0,0,0,1,0,0,1,1,NaN,NaN,NaN,NaN,NaN,Supply of troops and means.,Abastecimiento de tropas y medios e Suministro de tropas y médios.,Abastecimento.,NaN,0,0,0,1,1
2,"O termo é ""Tropa"".\nTropa tem definição: ""Termo coletivo que designa o pessoal de uma organizaçã...",3,Tropa,20230605,Termo,0,1,0,0,1,0,1,1,0,Termo coletivo que designa o pessoal de uma organização militar.,NaN,Organização militar e Missão.,NaN,NaN,Troop.,Tropa.,NaN,NaN,0,0,0,0,0
3,"O termo é ""Abono de faltas"".\nAbono de faltas tem definição: ""É a relevação da falta, de forma q...",4,Abono de faltas,20230605,Termo,1,1,1,0,0,0,1,0,0,"É a relevação da falta, de forma que o servidor não sofra qualquer desconto pelo dia não trabalh...","Faltas, Abono das faltas e Abono de ausências.",Inassiduidade habitual e Afastamento de pessoal.,Usado para servidores regidos pela CLT (art. 473).,NaN,NaN,NaN,NaN,NaN,0,0,0,1,1
4,"O termo é ""Abono de permanência em serviço"".\nAbono de permanência em serviço tem definição: ""O ...",5,Abono de permanência em serviço,20230605,Termo,0,1,1,0,0,0,1,0,0,O abono de permanência será pago ao servidor que tiver preenchido as exigências para aposentador...,"Abono de permanência, Adicional de permanência e Abono permanência.","Aposentadoria voluntária, Contribuição previdenciária e Regime Próprio de Previdência Social.",NaN,NaN,NaN,NaN,NaN,NaN,0,7,0,4,11


### Quantitative calculation

Tokenization loading

In [14]:
nome_modelo_ranking_pt = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_pt = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_pt
assert os.path.exists(nome_caminho_modelo_pt), f"Path para {nome_caminho_modelo_pt} não existe!"
tokenizador_pt_minilm = AutoTokenizer.from_pretrained(nome_caminho_modelo_pt)
nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")

nome_caminho_modelo_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_3b), f"Path para {nome_caminho_modelo_3b} não existe!"
tokenizador_pt_monot5_3b = AutoTokenizer.from_pretrained(nome_caminho_modelo_3b)
def retorna_num_tokens(parm_texto:str, parm_tokenizador:AutoTokenizer):
    return len(parm_tokenizador.tokenize(parm_texto))


/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [15]:
df_doc['LEN_TEXT_CHAR'] = df_doc['TEXT'].apply(len)
df_doc['LEN_TEXT_CHAR_LOG'] = round(np.log(df_doc['TEXT'].apply(len))).astype(int)
df_doc['NUM_WORD'] = df_doc['TEXT'].apply(lambda x: len(x.split()))
df_doc['NUM_TOKENS_MONOT5_3B'] = df_doc['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_monot5_3b)
df_doc['NUM_TOKENS_MINILM'] = df_doc['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_minilm)

In [16]:
df_doc.columns

Index(['TEXT', 'ID', 'NAME', 'DATE_REFERENCE', 'CLASS', 'HAS_SCOPE_NOTE', 'HAS_DEFINITION', 'HAS_SYNONYM', 'HAS_EXAMPLE', 'HAS_ENGLISH_TRANSLATION', 'HAS_SPECIALIZATION', 'HAS_RELATED_TERM',
       'HAS_SPANISH_TRANSLATION', 'HAS_GENERALIZATION', 'TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION',
       'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION', 'COUNT_INDEX_AREA', 'COUNT_INDEX_THEME', 'COUNT_INDEX_SUBTHEME', 'COUNT_INDEX_EXTRA', 'COUNT_INDEX_TOTAL',
       'LEN_TEXT_CHAR', 'LEN_TEXT_CHAR_LOG', 'NUM_WORD', 'NUM_TOKENS_MONOT5_3B', 'NUM_TOKENS_MINILM'],
      dtype='object')

In [17]:
%%time
data_index_json = []
for cnt, (index, row_doc) in tqdm(enumerate(df_doc.sort_values(by='ID').iterrows()), total=df_doc.shape[0]):
    #print(row_doc['TEXT_SPECIALIZATION'], type(row_doc['TEXT_SPECIALIZATION']), np.isnan(row_doc['TEXT_SPECIALIZATION']))
    #print(isinstance(row_doc['TEXT_SPECIALIZATION'],float))
    #print(row_doc['TEXT_ENGLISH_TRANSLATION'], type(row_doc['TEXT_ENGLISH_TRANSLATION'])) #, np.isnan(row_doc['TEXT_ENGLISH_TRANSLATION']))
    try:
        row_json = {
            'content': row_doc['TEXT'],
            'id': row_doc['ID'],
            'meta': {}
        }               
        for column_name in df_doc.columns:
            if column_name not in ['ID','TEXT']:
                # if np.isnan(row_doc[column_name]):
                if column_name.startswith('TEXT'):
                    if isinstance(row_doc[column_name],float):
                        row_json['meta'][column_name.lower()] = ""
                    else:
                        row_json['meta'][column_name.lower()] = row_doc[column_name]        
                else:
                    row_json['meta'][column_name.lower()] = row_doc[column_name]        
        
        data_index_json.append(row_json)
    except Exception as e:
        print('row', row_doc)
        print(e)
        continue


100%|██████████| 13252/13252 [00:01<00:00, 8699.59it/s]

CPU times: user 1.52 s, sys: 7.5 ms, total: 1.53 s
Wall time: 1.53 s


In [18]:
print(f"Total {len(data_index_json)}")

Total 13252


In [19]:
print(data_index_json[0])


{'content': 'O termo é "Abandono de cargo".\nAbandono de cargo tem definição: "Configura abandono de cargo a ausência intencional do servidor ao serviço por mais de trinta dias consecutivos.".\nAbandono de cargo tem nota de escopo: "Artigo 323 Código Penal: "Abandonar cargo público, fora dos casos permitidos em lei."".\nAbandono de cargo tem sinônimo: "Abandono de função" e "Abandono do cargo".\nAbandono de cargo tem termo relacionado: "Abandono de emprego", "Deserção", "Falta grave", "Função pública", "Demissão de servidor", "Inassiduidade habitual", "Cargo público", "Processo administrativo disciplinar" e "Pena disciplinar".\nAbandono de cargo tem tradução em espanhol: "Abandono de puesto".\nAbandono de cargo tem tradução em inglês: "Abandonment of office".', 'id': 1, 'meta': {'name': 'Abandono de cargo', 'date_reference': 20230605, 'class': 'Termo', 'has_scope_note': 1, 'has_definition': 1, 'has_synonym': 1, 'has_example': 0, 'has_english_translation': 1, 'has_specialization': 0, 'h

# Index creation

In [20]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

There are no index with name indir%


In [ ]:
# util_elastic.delete_index('indir_juris_tcu_index')

In [21]:
INDEX_NAME = 'indir_juris_tcu_index'

In [22]:
index = util_elastic.create_index(parm_index_name=INDEX_NAME, parm_data_carga_json=data_index_json,  parm_embedding_dim=1024)


before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 1: None

after write

Qtd de documentos 13252

Qtd de embeddings 0

Documento.id= 1: <Document: id=1, content='O termo é "Abandono de cargo".
Abandono de cargo tem definição: "Configura abandono de cargo a ausên...'>


In [24]:
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo
assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"


In [25]:
%%time
util_elastic.update_index_embedding_sts(parm_index=index, parm_path_model=nome_caminho_modelo)

Updating embeddings:   0%|          | 0/13252 [00:00<?, ? Docs/s]